In [1]:
# Download required Libraries
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 37.3 MB/s 
     |████████████████████████████████| 8.8 MB 93.7 MB/s 
     |████████████████████████████████| 198 kB 74.8 MB/s 
     |████████████████████████████████| 3.0 MB 68.7 MB/s 
     |████████████████████████████████| 98 kB 11.5 MB/s 
     |████████████████████████████████| 99 kB 12.4 MB/s 
     |████████████████████████████████| 1.2 MB 59.6 MB/s 
     |████████████████████████████████| 1.3 MB 72.9 MB/s 
     |████████████████████████████████| 46 kB 5.4 MB/s 
     |████████████████████████████████| 130 kB 104.3 MB/s 
     |████████████████████████████████| 3.4 MB 93.7 MB/s 
     |████████████████████████████████| 128 kB 107.3 MB/s 
     |████████████████████████████████| 1.6 MB 88.8 MB/s 
     |████████████████████████████████| 9.7 MB 51.6 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |█████████████████████████████

In [1]:
# Import Libraries
import ee
import geemap as gm

In [2]:
# Interactive Map
Map = gm.Map()
Map.add_basemap('HYBRID')
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=iE9Dh2tSoZ94J0vXQFUF8vZ9BzlEggtn9VsZD_hUFTM&tc=eo2kUoJX8aySnN7rauXuMohKCEZlwsEyGJj-57B-yUE&cc=DYRY61l-2h49DalQmgct32Z2E-dSm1-POAHEmRz8lEc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qjvRwulDebiO4R28BbVmS-n-HialFyb1x4ZQpj8uB__L7mYqaeirpI

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# **Classification Model - Supervised Classification**

In [32]:
# Satellite Data

# Point of Interest
point = ee.Geometry.Point([-122.4439, 37.7538])
# ee.Geometry.Point([-122.4439, 37.7538]) - CA
# ee.Geometry.Point([78.1194, 9.9254]) - Madurai

start_date = '2018-01-01'
end_date = '2018-12-31'

# Satellite Image
image = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(point)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 25))
    # .sort('CLOUD_COVER')
    .select('B[1-7]')
    # .median()
    .first()
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Sentinel-2 Image")

In [33]:
# Region of Interest

# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Rectangle([78.0202, 9.8239, 78.2207, 10.0066])
# [78.02024423140071, 9.823976326145358] [78.22074471968196, 10.006601373395881]
region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In [35]:
# To create Label

# Create Dynamic World land cover composite
landcover = gm.dynamic_world(region, start_date, end_date, return_type='hillshade')
Map.addLayer(landcover, {}, 'Land Cover')

# Add legend to the map
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
Map

Map(bottom=25636.0, center=[37.75334401310659, -122.44537353515626], controls=(WidgetControl(options=['positio…

In [36]:
# Training Dataset

points = landcover.sample(
    **{
        'region': image.geometry(),
        'scale': 500,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True, 
    }
)

Map.addLayer(points, {}, 'training', False)

In [37]:
print(points.size().getInfo())

4970


In [38]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-121.91261299607056, 36.95797024862996]}, 'id': '0', 'properties': {'vis-blue': 0.6173010468482971, 'vis-green': 0.4290657639503479, 'vis-red': 0.17993080615997314}}


In [39]:
# Training Classifier

# Bands for Prediction
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This property of the table stores the land cover labels.
label = 'label'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [40]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 35, 'B2': 202, 'B3': 265, 'B4': 154, 'B5': 127, 'B6': 146, 'B7': 140}}


In [ ]:
# Image Classification
result = image.select(bands).classify(trained)

# Display the Clusters with Random Colors
Map.addLayer(result.randomVisualizer(), {}, 'Classified Map')
Map

In [ ]:
# Render Categorical Map
class_values = landcover.get('landcover_class_values').getInfo()
class_values

In [ ]:
class_palette = landcover.get('landcover_class_palette').getInfo()
class_palette

In [16]:
land = result.set('classification_class_values', class_values)
land = land.set('classification_class_palette', class_palette)

Map.addLayer(land, {}, 'Land cover')

In [41]:
# Visualize the Results
print('Change Layer Opacity')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change Layer Opacity


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [42]:
# Map with legend
Map.add_legend(builtin_legend='Land Cover')
Map

The builtin legend must be one of the following: NLCD, ESA_WorldCover, ESRI_LandCover, Dynamic_World, NWI, MODIS/051/MCD12Q1, MODIS/006/MCD12Q1, GLOBCOVER, JAXA/PALSAR, Oxford, AAFC/ACI, COPERNICUS/CORINE/V20/100m, COPERNICUS/Landcover/100m/Proba-V/Global, USDA/NASS/CDL, ALOS_landforms


Map(bottom=25636.0, center=[37.75334401310659, -122.44262695312501], controls=(WidgetControl(options=['positio…

# **Reference Map Visualization**

In [43]:
# Visualizing Dynamic World Land Cover Data
# https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1?hl=en

Map = gm.Map()

region = ee.Geometry.BBox(-179, -89, 179, 89)
start_date = '2021-01-01'
end_date = '2022-01-01'

dw_class = gm.dynamic_world(region, start_date, end_date, return_type='class')
dw = gm.dynamic_world(region, start_date, end_date, return_type='hillshade')

dw_vis = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}

Map.addLayer(dw_class, dw_vis, 'DW Land Cover', False)
Map.addLayer(dw, {}, 'DW Land Cover Hillshade')

Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
Map.setCenter(-122.4439, 37.7538, 8)
Map

Map(center=[37.7538, -122.4439], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

# **Compare Two Maps - Classification and Reference**

In [44]:
# Compare Classified Map with Dynamic World

Map = gm.Map(center=[39.3322, -106.7349], zoom=10)

left_layer = gm.ee_tile_layer(landcover, {}, 'Land Cover')
right_layer = gm.ee_tile_layer(dw, {}, "Dynamic World Land Cover")

Map.split_map(left_layer, right_layer)

Map.add_legend(
    title="Classified Map", 
    builtin_legend='Classified_Map', 
    position='bottomleft'
)

Map.add_legend(
    title="Dynamic World Land Cover",
    builtin_legend='Dynamic_World',
    position='bottomright',
)
Map.setCenter(-122.4439, 37.7538, 8)

Map

The builtin legend must be one of the following: NLCD, ESA_WorldCover, ESRI_LandCover, Dynamic_World, NWI, MODIS/051/MCD12Q1, MODIS/006/MCD12Q1, GLOBCOVER, JAXA/PALSAR, Oxford, AAFC/ACI, COPERNICUS/CORINE/V20/100m, COPERNICUS/Landcover/100m/Proba-V/Global, USDA/NASS/CDL, ALOS_landforms


Map(center=[37.7538, -122.4439], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

# **Change Monitoring and Detection**

In [46]:
# Parameters

Map.add_basemap('HYBRID')
Map

# Region of Interest
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-89.7088, 42.9006, -89.0647, 43.2167)

Map.centerObject(region)

# Date Range
start_date = '2019-01-01'
end_date = '2021-12-31'

In [47]:
# Time Series
# return_type = hillshade, visualize, class, or probability

images = gm.dynamic_world_timeseries(
    region, start_date, end_date, return_type="class"
)

In [ ]:
# Labels and Legend

vis_params = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}
Map.addLayer(images.first(), vis_params, 'First image')
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
Map

In [49]:
Map.ts_inspector(images, left_vis=vis_params, date_format='YYYY')

In [50]:
Map = gm.Map()
Map.add_basemap('HYBRID')
Map.centerObject(region)

images = gm.dynamic_world_timeseries(
    region, start_date, end_date, return_type="hillshade"
)
Map.ts_inspector(images, date_format='YYYY')
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=Lay…